## Introduction

## Preliminary Exploratory data analysis

In [1]:
car_data <- read.table("data/car.data", sep = ",")
nrow(car_data)
colnames(car_data) = c("buying", "maint", "doors", "persons", "lug_boot", "safety", "class")
car_data

[1] 1728

buying,maint,doors,persons,lug_boot,safety,class
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
vhigh,vhigh,2,2,small,low,unacc
vhigh,vhigh,2,2,small,med,unacc
vhigh,vhigh,2,2,small,high,unacc
vhigh,vhigh,2,2,med,low,unacc
vhigh,vhigh,2,2,med,med,unacc
vhigh,vhigh,2,2,med,high,unacc
vhigh,vhigh,2,2,big,low,unacc
vhigh,vhigh,2,2,big,med,unacc
vhigh,vhigh,2,2,big,high,unacc


## Methods

## Expected outcomes and significance